In [1]:
using Distributions, Plots

In [2]:
candidate_n = 10 # Number of validator candidates
validator_n = 5 # Number of validator slots
nominator_n = 10; # Number of nominator candidates

In [3]:
# Nominator preferences, determined by their stake, preferred return and validators
function nomination_preference(ordered_candidates::Vector)
    l = length(ordered_candidates)
    rand(1:l, rand(1:l))
end

nomination_preference (generic function with 1 method)

In [37]:
candidate_stake(candidate_n::Int) = sort(rand(Geometric(0.01), candidate_n)) # Validators represented only by their stake, always ordered
nominator_stake(nominator_n::Int) = sort(rand(Geometric(0.01), nominator_n)) # Nominator stakes
nominator_interest(nominator_n::Int) = rand(Beta(10, 100), nominator_n) # Desired nominator interests

nominator_interest (generic function with 1 method)

In [38]:
# Total stake
total_stake(validators::Vector, nominations) = validators + sum(nominations, 2)[:]
inflation(stakes::Vector, rates::Vector) = (rates' * stakes)[1]
inflation(validators::Vector, nominations, rates::Vector) = inflation(total_stake(validators, nominations), rates)
# Threshold
threshold(validator_n::Int, candidates::Vector, nominations) = minimum(sort(sum(nominations, 2)[:]+candidates)[end-validator_n+1:end])

threshold (generic function with 1 method)

In [52]:
struct World
    validator_n::Int
    candidate_n::Int
    candidates::Vector{Int}
    nominator_n::Int
    nominators::Vector{Int}
    interests::Vector{Float64}
    preferences::Vector{Vector{Int}}
end
function World(validator_n::Int, candidate_n::Int, nominator_n::Int)
    candidates = candidate_stake(candidate_n)
    World(
        validator_n,
        candidate_n,
        candidates,
        nominator_n,
        nominator_stake(nominator_n),
        nominator_interest(nominator_n),
        [nomination_preference(candidates) for _ in 1:nominator_n]
    )
end

World

In [67]:
function sample(validator_n::Int, candidate_n::Int, nominator_n::Int, strat::Function, sample_n::Int)
    infs = zeros(sample_n)
    ths = zeros(sample_n)
    for s in 1:sample_n
        world = World(validator_n, candidate_n, nominator_n)
        rates, nominations = strat(world)
        available_stake = sum(world.nominators)+sum(world.candidates)
        infs[s] = inflation(world.candidates, nominations, rates)/available_stake
        ths[s] = threshold(validator_n, world.candidates, nominations)/available_stake
    end
    (infs, ths)
end

sample (generic function with 1 method)

In [68]:
function brute_strat(w::World)
    rates = rand(Beta(10, 100), w.candidate_n)
    nominations = zeros(w.candidate_n, w.nominator_n)
    subset = rand(1:w.candidate_n, w.validator_n)
    for n in 1:w.nominator_n
        preference = filter(p -> p in subset, w.preferences[n])
        c = length(preference)
        for p in preference
            if rates[p] >= w.interests[n]
                nominations[p, n] += w.nominators[n]/c
            end
        end
    end
    (rates, nominations)
end

brute_strat (generic function with 1 method)

In [69]:
function adarates_strat(w::World)
    rates = zeros(w.candidate_n)
    nominations = zeros(w.candidate_n, w.nominator_n)
    subset = rand(1:w.candidate_n, w.validator_n)
    for n in 1:w.nominator_n
        preference = filter(p -> p in subset, w.preferences[n])
        c = length(preference)
        for p in preference
            if w.interests[n] > rates[p]
                rates[p] = w.interests[n]
            end
            nominations[p, n] += w.nominators[n]/c
        end
    end
    (rates, nominations)
end

adarates_strat (generic function with 1 method)

In [76]:
function th_strat(w::World)
    t = (sum(w.nominators)+sum(w.candidates[1:w.validator_n]))/w.validator_n
    rates = zeros(candidate_n)
    nominations = zeros(candidate_n, nominator_n)
    for n in 1:nominator_n
        remaining_nomination = w.nominators[n]
        for p in w.preferences[n]
            existing_stake = sum(nominations[p, :]) + w.candidates[p]
            diff = t - existing_stake
            if diff > 0
                if w.interests[n] > rates[p]
                    rates[p] = w.interests[n]
                end
                if diff >= remaining_nomination
                    nominations[p, n] += remaining_nomination
                    break
                else
                    nominations[p, n] += diff
                    remaining_nomination -= diff
                end
            end
        end
    end
    (rates, nominations)
end

th_strat (generic function with 1 method)

In [83]:
# X is inflation, Y is threshold
scatter(sample(validator_n, candidate_n, nominator_n, th_strat, 1000), label = "th", xaxis = "Inflation", yaxis = "Threshold")
scatter!(sample(validator_n, candidate_n, nominator_n, adarates_strat, 1000), label = "adarates")
scatter!(sample(validator_n, candidate_n, nominator_n, brute_strat, 1000), label = "brute")